## Importing the relevant libraries

In [ ]:
!pip install wget --quiet


In [ ]:
!pip install tensorflow==2.10.0 --quiet

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, wget
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, PrecisionRecallDisplay, RocCurveDisplay
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from wordcloud import WordCloud
wget.download("https://raw.githubusercontent.com/yogawicaksana/helper_prabowo/main/helper_prabowo_ml.py",out="helper_prabowo_ml.py")
from helper_prabowo_ml import clean_html, punct, remove_digits, remove_links, remove_special_characters, remove_, removeStopWords, lower, email_address, non_ascii
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

2025-04-02 08:05:29.526244: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-02 08:05:30.745443: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-04-02 08:05:30.745471: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-04-02 08:05:30.880616: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-02 08:05:33.535272: W tensorflow/stream_executor/platform/de

## Loading the train, test and evaluation datasets

In [ ]:
train_df = pd.read_csv("data/train (2).csv",delimiter=';')
train_df.drop('Unnamed: 0',axis=1,inplace=True)
train_df.head(5)

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.isna().sum()

In [ ]:
train_df.duplicated().sum()

In [ ]:
train_df = train_df.drop_duplicates()
train_df.shape

In [ ]:
test_df = pd.read_csv('data/test (1).csv',delimiter=';')
test_df.drop('Unnamed: 0',axis=1,inplace=True)
test_df.head(3)

In [ ]:
test_df.shape

In [ ]:
test_df.isnull().sum()

In [ ]:
test_df.duplicated().sum()

In [ ]:
evaluation_df = pd.read_csv("data/evaluation.csv",delimiter=';')
evaluation_df.drop('Unnamed: 0',axis=1,inplace=True)
evaluation_df.head()

In [ ]:
evaluation_df.shape

In [ ]:
evaluation_df.isna().sum()

In [ ]:
evaluation_df.duplicated().sum()

## Data Exploration

In [ ]:
train_df['num_words'] = train_df.text.apply(len)
train_df.num_words.describe()

In [ ]:
plt.figure(figsize=(10,5))
sns.distplot(train_df['num_words']);

In [8]:
max_len = 120

In [ ]:
plt.figure(figsize=(12,6))
wc = WordCloud(width=600,height=300,random_state=101).generate(' '.join(train_df.text))
plt.imshow(wc)
plt.title('Word Cloud of News Text',fontsize=25,pad=20,fontweight='bold',color='sienna')
plt.show();

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='label',data=train_df);

The target fake news label is relatively balanced.

## Text Preprocessing

In [ ]:
def preprocess_text(data,col):
    data[col] = data[col].apply(func=lower)
    data[col] = data[col].apply(func=clean_html)
    data[col] = data[col].apply(func=non_ascii)
    data[col] = data[col].apply(func=email_address)
    data[col] = data[col].apply(func=punct)
    data[col] = data[col].apply(func=removeStopWords)
    data[col] = data[col].apply(func=remove_)
    data[col] = data[col].apply(func=remove_digits)
    data[col] = data[col].apply(func=remove_links)
    data[col] = data[col].apply(func=remove_special_characters)
    return data
    

In [ ]:
preprocessed_train = preprocess_text(train_df,'text')
preprocessed_test = preprocess_text(test_df,'text')
preprocessed_eval = preprocess_text(evaluation_df,'text')

## Downloading pretrained Tokenizer and BERT model from Hugging Face

In [5]:
model_name = 'mrm8488/bert-tiny-finetuned-fake-news-detection'
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert = TFAutoModelForSequenceClassification.from_pretrained(model_name,from_pt=True)

2025-04-02 08:05:52.362831: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-04-02 08:05:52.364488: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2025-04-02 08:05:52.364552: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (artefact-de-vm-wissem0072): /proc/driver/nvidia/version does not exist
2025-04-02 08:05:52.367576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSeque

## Performing text tokenization

In [ ]:
X_train = tokenizer(text=preprocessed_train.text.tolist(),
                   max_length=max_len,
                   padding=True,
                   truncation=True,
                   add_special_tokens=True,
                   return_token_type_ids=False,
                   return_attention_mask=True,
                   return_tensors='tf',
                   verbose=1)

X_test = tokenizer(text=preprocessed_test.text.tolist(),
                  max_length=max_len,
                  padding=True,
                  truncation=True,
                  add_special_tokens=True,
                  return_token_type_ids=False,
                  return_attention_mask=True,
                  return_tensors='tf',
                  verbose=1)

X_eval = tokenizer(text=preprocessed_eval.text.tolist(),
                  max_length=max_len,
                  padding=True,
                  truncation=True,
                  add_special_tokens=True,
                  return_token_type_ids=False,
                  return_attention_mask=True,
                  return_tensors='tf',
                  verbose=1)



In [ ]:
print(X_train)


## Encoding the target label classes

In [ ]:
scaler = LabelEncoder()
train_df.label = scaler.fit_transform(train_df.label)
test_df.label = scaler.transform(test_df.label)
evaluation_df.label = scaler.transform(evaluation_df.label)

## Model Training & Evaluation

In [ ]:
input_ids = Input(shape=(max_len,),dtype=tf.int32,name='input_ids')
attention_mask = Input(shape=(max_len,),dtype=tf.int32,name='attention_mask')

In [ ]:
embeddings = bert(input_ids,attention_mask=attention_mask)[0] # 0 -> final hidden state, 1 -> pooling output
output = Flatten()(embeddings)
output = Dense(units=512,activation='relu')(output)
output = BatchNormalization()(output)
output = Dropout(0.3)(output)
output = Dense(units=256,activation='relu')(output)
output = BatchNormalization()(output)
output = Dropout(0.2)(output)
output = Dense(units=128,activation='relu')(output)
output = BatchNormalization()(output)
output = Dropout(0.15)(output)
output = Dense(units=1,activation='sigmoid')(output)
model = Model(inputs=[input_ids,attention_mask],outputs=output)
model.layers[2].trainable = False
model.summary()

In [ ]:
plot_model(model,to_file='model.png',show_shapes=True,dpi=100)

In [ ]:
optimizer = Adam(learning_rate=4e-5,epsilon=1e-7,decay=1e-2,clipnorm=1.0)
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
def step_decay(epoch, lr):
    drop_rate = 0.5
    step_size = 10
    return lr * drop_rate ** (epoch // step_size)

In [ ]:
es = EarlyStopping(monitor='val_accuracy',mode='max',patience=10,restore_best_weights=True,verbose=1)
lrs = LearningRateScheduler(step_decay,verbose=1)
mc = ModelCheckpoint(filepath='fake_news_classifier.keras',monitor='val_accuracy',save_best_only=True,mode='max',save_freq='epoch',verbose=1)

r = model.fit(x={'input_ids': X_train['input_ids'], 'attention_mask': X_train['attention_mask']},
             y=train_df.label,
             epochs=20,
             batch_size=256,
             callbacks=[es,lrs,mc],
             validation_data=({'input_ids': X_test['input_ids'], 'attention_mask': X_test['attention_mask']},test_df.label)
             )

## Visualizing model performance

In [ ]:
plt.plot(r.history['loss'],'r',label='train loss')
plt.plot(r.history['val_loss'],'b',label='test loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Binary Crossentropy Loss')
plt.title('Loss Graph')
plt.legend()
plt.show();

In [ ]:
plt.plot(r.history['accuracy'],'r',label='train accuracy')
plt.plot(r.history['val_accuracy'],'b',label='test accuracy')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.show();

## Evaluating model on the test dataset

In [ ]:
loss, acc = model.evaluate({'input_ids': X_test['input_ids'], 'attention_mask': X_test['attention_mask']},test_df.label)
print("Test Binary Crossentropy Loss:", round(loss,2))
print("Test Accuracy:", round(acc*100,2))

In [ ]:
test_predictions = model.predict({'input_ids': X_test['input_ids'], 'attention_mask': X_test['attention_mask']})
test_predictions = (test_predictions > 0.5).astype('float')
print("Confusion Matrix:")
print(confusion_matrix(test_df.label,test_predictions))
print("Classification Report:")
print(classification_report(test_df.label,test_predictions))
ConfusionMatrixDisplay.from_predictions(test_df.label,test_predictions)
plt.show()
PrecisionRecallDisplay.from_predictions(test_df.label,test_predictions)
plt.show()
RocCurveDisplay.from_predictions(test_df.label,test_predictions)
plt.show()

## Model Inference

In [ ]:
test_predictions = model.predict({'input_ids': X_eval['input_ids'], 'attention_mask': X_eval['attention_mask']})
test_predictions = (test_predictions > 0.5).astype('float')
print("Confusion Matrix:")
print(confusion_matrix(evaluation_df.label,test_predictions))
print("Classification Report:")
print(classification_report(evaluation_df.label,test_predictions))
ConfusionMatrixDisplay.from_predictions(evaluation_df.label,test_predictions)
plt.show()
PrecisionRecallDisplay.from_predictions(evaluation_df.label,test_predictions)
plt.show()
RocCurveDisplay.from_predictions(evaluation_df.label,test_predictions)
plt.show()

Bravo! The model has performed incredibly well by achieving an astonishing accuracy of more than 97% on the evaluation holdout set.

Don't forget to upvote my notebook if you like it! If you have any feedback, kindly share it in the comments section below.

In [ ]:
tokenizer.save_pretrained(save_directory='tokenizer/') # Saving the pretrained tokenizer

In [ ]:
The error occurs because the directory `tokenizer/` does not contain the necessary configuration files (`config.json`) or the correct model files required to load a model using `TFAutoModelForSequenceClassification`. The `tokenizer.save_pretrained()` method only saves the tokenizer, not the model itself.

To resolve this issue, you need to save the model properly and then load it. Here's how you can fix it:

1. Save the model:
    ```python
    bert.save_pretrained(save_directory='model/')
    ```

2. Load the model:
    ```python
    loaded_model = TFAutoModelForSequenceClassification.from_pretrained('model/')
    ```

Make sure you save the model using the `save_pretrained()` method of the model object (`bert` in this case) and not the tokenizer.

In [ ]:
bert.save_pretrained(save_directory='model/')

In [16]:
loaded_model = TFAutoModelForSequenceClassification.from_pretrained('model/')


Some layers from the model checkpoint at model/ were not used when initializing TFBertForSequenceClassification: ['dropout_7']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
def predict_text(text):
    tokenized_input = tokenizer(text=text,
                                max_length=max_len,
                                padding=True,
                                truncation=True,
                                add_special_tokens=True,
                                return_token_type_ids=False,
                                return_attention_mask=True,
                                return_tensors='tf',
                                verbose=1)
    prediction = loaded_model.predict({'input_ids': tokenized_input['input_ids'], 'attention_mask': tokenized_input['attention_mask']})
    return (prediction.logits > 0.5).astype('float')
prediction_final = predict_text("Ryan has never made any secret about his desire to welcome immigrants to America. How many will live with him in his large family home in the confines of his fenced in compound? A recent Breitbart News investigation reveals that Rep. Paul Ryan (R-WI), who has a two-decade-long history of promoting open borders immigration policies, seems to support border fences for himself, even as he denies the American people those same protections.While Paul Ryan s omnibus spending bill does not provide funding for the mandatory completion of a 700-mile double-layer border fence that Congress promised the American people nearly a decade ago when it passed the 2006 Secure Fence Act, Paul Ryan has constructed a fence around his property.As Breitbart News s photographic documentation reveals, Ryan s home is surrounded by a tall border fence reinforced by equally high bushes  ensuring both privacy and security. Moreover, the fence is manned by an on-duty agent who guards his property s perimeter. Upon even the slightest appearance of any unusual activity  such as a 5 2  female taking a photograph of the fence  Ryan s border agent will deploy into action to ensure the perimeter s sovereignty.Over the course of the past six months, there has been heightened national focus on Americans  desire for a border fence following the GOP presidential frontrunners  call for a border wall. A Rasmussen Reports survey released in August of this year found that likely Republican voters, by greater than a 4-1 margin, support Donald Trump s plan to build a border wall (70 percent vs. 17 percent). Amongst all likely voters, a majority (51 percent) support building the border wall.Congress promised the American people a 700-mile border fence in the 2006 Secure Fence Act. However, funding for the project was subsequently gutted and, as a result, construction was never completed.While Paul Ryan s $1.1 trillion year-end omnibus spending package was able to allocate funding for immigration programs that benefit foreign nationals  such as federal grants for lawless sanctuary cities and the U.S. resettlement tens of thousands of refugees  his bill does not require an allotment of funds be spent on the completion of the 700-mile-long fence that the American voters were promised.Sen. Jeff Sessions (R-AL)has previously highlighted the hypocrisy of immigration expansionists who surround their homes with border fences and monitor who comes on their property but do not apparently believe the American people deserve the same protections. Sessions pointed specifically to open borders advocate Mark Zuckerberg who, according to reports, spent $30 million buying the surrounding four homes around his own property in order to get  a little more privacy. Sessions said,  Well, the  masters of the universe  are very fond of open borders as long as these open borders don t extend to their gated compounds and fenced-off estates. On previous occasions, Ryan has repeatedly suggested that the American people are not entitled to discriminate against who enters their country on a visa. When Sean Hannity asked Paul Ryan about whether or not he would support curbs to Muslim immigration, Paul Ryan declared,  That s not who we are. However, Ryan s fence ensures that no refugees will be able to enter his property without his permission  even as U.S. communities are not able to make any such restrictions.In 2013, when Ryan traveled with Rep. Luis Gutierrez (D-IL)to stump for Sen. Marco Rubio s (R-FL) immigration agenda, Ryan declared that America  is more than our borders  and that the U.S. ought to have an  open door  system where foreign nationals can come and go as they please. America is more than just a country,  Ryan said.  It s more than Chicago, or Wisconsin. It s more than our borders. America is an idea. It s a very precious idea. Ryan disparaged Americans who opposed large-scale immigration, characterizing that attitude as ignorant, declaring that throughout American history,  Each wave [of immigration] is met with some ignorance, is met with some resistance. Ryan said,  We want to have a system where people can come here and work  go back and forth if they want to  so that we have an open door to the people who want to come and contribute to our country, who want to come and make a difference in their families  lives, and our economy.  Via: Breitbart News")
print(prediction_final)   

1/1 [==============================] - 0s 36ms/step
[[1. 0.]]
